# Lab 2: Utility-Scale Layer Fidelity Experiment

Samantha Barron, Haimeng Zhang

In [28]:
%set_env QXToken=eafe6b19915883c206e07330426ebeec6281dd5f8cb1462262da6e8c2c76c89c315fa361276dc4e8c2679cd1bc2031cb73c747fea64ab964e0e785ca2f912232
# Make sure there is no space between the equal sign
# and the beginning of your token
# Make sure you do NOT ADD QUOTATION MARKS!!!!ken=Your_Quantum_API_Token_goes_here

env: QXToken=eafe6b19915883c206e07330426ebeec6281dd5f8cb1462262da6e8c2c76c89c315fa361276dc4e8c2679cd1bc2031cb73c747fea64ab964e0e785ca2f912232


In [29]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Save an IBM Cloud account on disk
token ='eafe6b19915883c206e07330426ebeec6281dd5f8cb1462262da6e8c2c76c89c315fa361276dc4e8c2679cd1bc2031cb73c747fea64ab964e0e785ca2f912232'

QiskitRuntimeService.save_account(channel="ibm_quantum", token=token,overwrite=True)

In [30]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()

In [31]:
from qiskit_ibm_provider import IBMProvider

# Load your IBM Quantum account
provider = IBMProvider()

# Choose the backend
BACKEND = provider.get_backend('ibm_osaka')

# Define the path of qubits
PATH_ANSWER: list[int] = [0, 1, 2, 3, 4]

# Print backend information
print(f"Backend: {BACKEND.name}")
print(f"Number of qubits: {BACKEND.configuration().num_qubits}")
print(f"Chosen qubit path: {PATH_ANSWER}")

# Verify that the path is valid
coupling_map = BACKEND.configuration().coupling_map
for i in range(len(PATH_ANSWER) - 1):
    assert [PATH_ANSWER[i], PATH_ANSWER[i+1]] in coupling_map or [PATH_ANSWER[i+1], PATH_ANSWER[i]] in coupling_map, f"Qubits {PATH_ANSWER[i]} and {PATH_ANSWER[i+1]} are not connected"

print("Path verification successful!")

Backend: ibm_osaka
Number of qubits: 127
Chosen qubit path: [0, 1, 2, 3, 4]
Path verification successful!


In [32]:
from qc_grader.challenges.qgss_2024 import grade_lab2_ex1
grade_lab2_ex1(PATH_ANSWER, BACKEND)

Submitting your answer. Please wait...
Nice work, all your answers look correct!


In [33]:

# Retrieve the coupling map from the backend
coupling_map = BACKEND.configuration().coupling_map

# After seeing the coupling map, we'll manually define a valid path and layers
# For now, let's leave these as empty and fill them in after seeing the coupling map
PATH_ANSWER = []
LAYER_1_ANSWER = []
LAYER_2_ANSWER = []

# Print backend information
print(f"\nBackend: {BACKEND.name}")
print(f"Number of qubits: {BACKEND.configuration().num_qubits}")

# The rest of the verification code will be added after we define the path and layers

print("Please provide a valid PATH_ANSWER, LAYER_1_ANSWER, and LAYER_2_ANSWER based on the printed coupling map.")


Backend: ibm_osaka
Number of qubits: 127
Please provide a valid PATH_ANSWER, LAYER_1_ANSWER, and LAYER_2_ANSWER based on the printed coupling map.


In [34]:
# grade part 2
from qc_grader.challenges.qgss_2024 import grade_lab2_ex2
grade_lab2_ex2(LAYER_1_ANSWER, LAYER_2_ANSWER, PATH_ANSWER, BACKEND)

Submitting your answer. Please wait...
Nice work, all your answers look correct!


In [35]:
from qiskit import QuantumCircuit
from qiskit_ibm_provider import IBMProvider

# Load your IBM Quantum account
provider = IBMProvider()

# Choose the backend
BACKEND = provider.get_backend('ibm_osaka')

# Retrieve the coupling map and basis gates from the backend
coupling_map = BACKEND.configuration().coupling_map
basis_gates = BACKEND.configuration().basis_gates


# Define LAYER_1_ANSWER and LAYER_2_ANSWER based on the coupling map

LAYER_1_ANSWER = [(1, 2), (5, 8)]
LAYER_2_ANSWER = [(2, 3), (3, 5)]

# Choose the appropriate two-qubit Clifford gate
if 'ecr' in basis_gates:
    GATE_NAME = 'ecr'
elif 'cz' in basis_gates:
    GATE_NAME = 'cz'
elif 'cx' in basis_gates:
    GATE_NAME = 'cx'
else:
    raise ValueError("No suitable two-qubit Clifford gate found in basis gates")

print(f"\nChosen two-qubit Clifford gate: {GATE_NAME}")

# Function to create a circuit for a given layer
def create_circuit(layer, num_qubits):
    circuit = QuantumCircuit(num_qubits)
    for edge in layer:
        if GATE_NAME == 'ecr':
            circuit.ecr(edge[0], edge[1])
        elif GATE_NAME == 'cz':
            circuit.cz(edge[0], edge[1])
        elif GATE_NAME == 'cx':
            circuit.cx(edge[0], edge[1])
    return circuit

# Create the circuits
num_qubits = max(max(qubit for edge in LAYER_1_ANSWER for qubit in edge),
                 max(qubit for edge in LAYER_2_ANSWER for qubit in edge)) + 1

CIRC_1_ANSWER = create_circuit(LAYER_1_ANSWER, num_qubits)
CIRC_2_ANSWER = create_circuit(LAYER_2_ANSWER, num_qubits)



Chosen two-qubit Clifford gate: ecr


In [36]:
# grade part 3
from qc_grader.challenges.qgss_2024 import grade_lab2_ex3
grade_lab2_ex3(CIRC_1_ANSWER, LAYER_1_ANSWER, GATE_NAME, BACKEND)
grade_lab2_ex3(CIRC_2_ANSWER, LAYER_2_ANSWER, GATE_NAME, BACKEND)

Submitting your answer. Please wait...
Nice work, all your answers look correct!
Submitting your answer. Please wait...
Nice work, all your answers look correct!


In [37]:
import numpy as np

from qiskit.primitives.containers.bindings_array import BindingsArray

def get_clifford_rz_samples(
    circ: QuantumCircuit,
    num_samples: int,
    rng: np.random.Generator,
) -> BindingsArray:
    """Uniformly sample the Rz parameters in a `QuantumCircuit`
    from -pi, -pi/2, 0, +pi/2, +pi.
    """

    _allowed_ops = {'rz', 'sx', 'sxdg', 'ecr', 'barrier', 'measure'}
    if (_other := set(circ.count_ops().keys()) - _allowed_ops):
        raise ValueError(f"Circuit must only contain ops: {_allowed_ops}, got {_other}")

    sampled_pars = rng.integers(-2, 3, size=(num_samples, len(circ.parameters))) * (np.pi/2)

    return BindingsArray({
        tuple(circ.parameters): sampled_pars
    })

In [38]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.primitives.containers.bindings_array import BindingsArray
from qiskit_ibm_provider import IBMProvider

# Load your IBM Quantum account
provider = IBMProvider()

# Choose the backend
BACKEND = provider.get_backend('ibm_osaka')

# Retrieve the coupling map and basis gates from the backend
coupling_map = BACKEND.configuration().coupling_map
basis_gates = BACKEND.configuration().basis_gates

print("Coupling map:")
for edge in coupling_map:
    print(edge)

print("\nBasis gates:")
print(basis_gates)

# Define LAYER_1_ANSWER and LAYER_2_ANSWER based on the coupling map
LAYER_1_ANSWER = [(1, 2), (5, 8)]
LAYER_2_ANSWER = [(2, 3), (3, 5)]

# Choose the appropriate two-qubit Clifford gate
if 'ecr' in basis_gates:
    GATE_NAME = 'ecr'
elif 'cz' in basis_gates:
    GATE_NAME = 'cz'
elif 'cx' in basis_gates:
    GATE_NAME = 'cx'
else:
    raise ValueError("No suitable two-qubit Clifford gate found in basis gates")

print(f"\nChosen two-qubit Clifford gate: {GATE_NAME}")

# Function to create a circuit for a given layer
def create_circuit(layer, num_qubits):
    circuit = QuantumCircuit(num_qubits)
    for edge in layer:
        if GATE_NAME == 'ecr':
            circuit.ecr(edge[0], edge[1])
        elif GATE_NAME == 'cz':
            circuit.cz(edge[0], edge[1])
        elif GATE_NAME == 'cx':
            circuit.cx(edge[0], edge[1])
    return circuit

# Create the circuits
num_qubits = max(max(qubit for edge in LAYER_1_ANSWER for qubit in edge),
                 max(qubit for edge in LAYER_2_ANSWER for qubit in edge)) + 1

CIRCUIT_1_ANSWER = create_circuit(LAYER_1_ANSWER, num_qubits)
CIRCUIT_2_ANSWER = create_circuit(LAYER_2_ANSWER, num_qubits)

# Define the eplg_circuit function
def eplg_circuit(num_qubits, depth, layer, qubits):
    circuit = QuantumCircuit(num_qubits)
    for _ in range(depth):
        for edge in layer:
            if GATE_NAME == 'ecr':
                circuit.ecr(edge[0], edge[1])
            elif GATE_NAME == 'cz':
                circuit.cz(edge[0], edge[1])
            elif GATE_NAME == 'cx':
                circuit.cx(edge[0], edge[1])
    return circuit

# Create example_circuit_2
example_circuit_2 = eplg_circuit(
    num_qubits=BACKEND.configuration().num_qubits,
    depth=4,
    layer=LAYER_2_ANSWER,
    qubits=PATH_ANSWER,
)

# Define the get_clifford_rz_samples function
def get_clifford_rz_samples(
    circ: QuantumCircuit,
    num_samples: int,
    rng: np.random.Generator,
) -> BindingsArray:
    """Uniformly sample the Rz parameters in a `QuantumCircuit`
    from -pi, -pi/2, 0, +pi/2, +pi.
    """

    _allowed_ops = {'rz', 'sx', 'sxdg', 'ecr', 'barrier', 'measure'}
    if (_other := set(circ.count_ops().keys()) - _allowed_ops):
        raise ValueError(f"Circuit must only contain ops: {_allowed_ops}, got {_other}")

    sampled_pars = rng.integers(-2, 3, size=(num_samples, len(circ.parameters))) * (np.pi/2)

    return BindingsArray({
        tuple(circ.parameters): sampled_pars
    })

# Use the example_circuit_2 to create ARRAY_1_ANSWER
num_samples = 10
ARRAY_1_ANSWER = get_clifford_rz_samples(
    circ=example_circuit_2,
    num_samples=num_samples,
    rng=np.random.default_rng(42)
)

print("ARRAY_1_ANSWER:")
print(ARRAY_1_ANSWER)

# Similarly, for CIRCUIT_2_ANSWER
ARRAY_2_ANSWER = get_clifford_rz_samples(
    circ=CIRCUIT_2_ANSWER,
    num_samples=num_samples,
    rng=np.random.default_rng(43)  # Using a different seed for variety
)

print("\nARRAY_2_ANSWER:")
print(ARRAY_2_ANSWER)

Coupling map:
[1, 0]
[2, 1]
[2, 3]
[3, 4]
[5, 4]
[6, 5]
[6, 7]
[7, 8]
[8, 16]
[9, 8]
[9, 10]
[11, 10]
[12, 11]
[13, 12]
[14, 0]
[14, 18]
[15, 4]
[16, 26]
[17, 12]
[17, 30]
[18, 19]
[20, 19]
[21, 20]
[21, 22]
[22, 15]
[23, 22]
[24, 23]
[24, 25]
[24, 34]
[25, 26]
[27, 26]
[27, 28]
[28, 29]
[28, 35]
[30, 29]
[31, 30]
[31, 32]
[33, 20]
[35, 47]
[36, 32]
[37, 38]
[37, 52]
[39, 33]
[39, 38]
[40, 39]
[40, 41]
[42, 41]
[42, 43]
[43, 34]
[43, 44]
[45, 44]
[45, 46]
[45, 54]
[46, 47]
[48, 47]
[48, 49]
[49, 50]
[50, 51]
[51, 36]
[52, 56]
[53, 41]
[55, 49]
[57, 56]
[57, 58]
[58, 71]
[59, 58]
[59, 60]
[60, 53]
[61, 60]
[61, 62]
[62, 72]
[63, 62]
[64, 54]
[64, 63]
[65, 64]
[66, 65]
[66, 73]
[67, 66]
[68, 55]
[68, 67]
[68, 69]
[69, 70]
[71, 77]
[74, 70]
[75, 76]
[76, 77]
[78, 77]
[78, 79]
[79, 80]
[81, 72]
[81, 80]
[81, 82]
[83, 82]
[84, 83]
[85, 73]
[85, 84]
[86, 85]
[87, 86]
[87, 88]
[87, 93]
[88, 89]
[89, 74]
[90, 75]
[90, 94]
[91, 79]
[92, 83]
[92, 102]
[93, 106]
[94, 95]
[96, 95]
[96, 97]
[96, 10

In [39]:
# grade part 4
from qc_grader.challenges.qgss_2024 import grade_lab2_ex4
grade_lab2_ex4(ARRAY_1_ANSWER, example_circuit_1, num_samples)

Submitting your answer. Please wait...
Nice work, all your answers look correct! 
 Notice that the shape of the BindingsArray is [10], and there are {num_parameters} parameters.
